### 0. download the dataset

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-21 16:22:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.72MB/s    in 0.6s    

2025-02-21 16:22:41 (1.72 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
print(f"Length of dataset in characters: {len(text)}")
print("\nFirst 1000 characters:\n")
print(text[:1000])

Length of dataset in characters: 1115394

First 1000 characters:

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods k

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))
print(f"\nVocab size: {vocab_size}")


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z

Vocab size: 65


### 1. tokenize the text 

In [5]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# split the data into train and val
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target i

### implement the bigram language model
start from the simpliest model 

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(loss) 
print(logits.shape)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(f"step {steps} loss: {loss.item()}")

step 0 loss: 4.692410945892334
step 1 loss: 4.664144515991211
step 2 loss: 4.765714645385742
step 3 loss: 4.70655632019043
step 4 loss: 4.5956573486328125
step 5 loss: 4.7101240158081055
step 6 loss: 4.713661193847656
step 7 loss: 4.686909198760986
step 8 loss: 4.700076103210449
step 9 loss: 4.718283653259277
step 10 loss: 4.715603351593018
step 11 loss: 4.684308052062988
step 12 loss: 4.745601177215576
step 13 loss: 4.735717296600342
step 14 loss: 4.666238784790039
step 15 loss: 4.58615255355835
step 16 loss: 4.714625835418701
step 17 loss: 4.671982765197754
step 18 loss: 4.715047359466553
step 19 loss: 4.74489164352417
step 20 loss: 4.630162715911865
step 21 loss: 4.707578182220459
step 22 loss: 4.670665740966797
step 23 loss: 4.582583427429199
step 24 loss: 4.739546298980713
step 25 loss: 4.674807071685791
step 26 loss: 4.805595874786377
step 27 loss: 4.749917507171631
step 28 loss: 4.691989421844482
step 29 loss: 4.604404926300049
step 30 loss: 4.721841335296631
step 31 loss: 4.741

In [14]:
# after training, we have a much better result from the model
print(decode(m.generate(idx = torch.zeros((1, 1), 
                                          dtype=torch.long), 
                                          max_new_tokens=300)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


### implementing the self attention block

In [15]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [16]:
# for self attention, we want to get the current token and all past tokens (not the future tokens)
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        # take average of all previous tokens up to the current token
        xbow[b,t] = torch.mean(xprev, dim=0) # (B, T, C)

In [17]:
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x  # (B, T, C)

In [18]:
print(x[0])

print(xbow[0])
# first row is same
# second row is average of first two tokens
# third row is average of first three tokens
# ...

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [19]:
# this method would be very inefficient for large contexts
# lets use matrix multiplication to do this more efficiently
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
a = a/torch.sum(a, dim=1, keepdim=True)
b= torch.randint(0,10,(3,2)).float()
c= a @ b

print("a=",a)
print("---")
print("b=",b)
print("---")
print("c=",c)


a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [20]:
# scale dot product attention
torch.manual_seed(42)
B, T, C = 4, 8, 2
x = torch.randn((B, T, C))

head_size = 16 
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)

wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

print(out.shape)

torch.Size([4, 8, 2])


In [21]:
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

count of notes about the attention mechanism
- the implementation above is a "self-attention"decoder block where the past and current tokens are fused to get the value 
- for an encoder block, we can simply remove the masked_fill so the entire context is used to get the value 
- self attention just means that the query, key, and value come from the same source i.e. x
- for cross attention, the query and key comes from a seperate encoder block (which is the gpt3 encoder decoder architecture)
- scaled attention divides wei by the sqrt of the head size 
- why? 

In [22]:
print(wei.var())
wei = q @ k.transpose(-2, -1) * head_size**-0.5
print(wei.var())2

tensor(0.0273)
tensor(0.3190, grad_fn=<VarBackward0>)


In [ ]:
# when the head size is large, the variance of the wei is too large and the softmax will saturate
# so we divide by the sqrt of the head size to normalize the variance
# this is a hack to make the training more stable

two hacks for training deep neural networks: 
- residual connections: add a seperate pathway for the gradients to flow through, this allows the residual blocks to learn only the residual (difference) between input x and output H(x)
- layer normalization: 

In [2]:
# here we focus on layer normalization
# lets start from the batch normalization code
import torch
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.9):
        self.eps = eps
        self.momentum = momentum
        self.training = True # different behavior based on training or validation 
        self.gamma = torch.ones((dim))
        self.beta = torch.zeros((dim))
        self.mean_running = torch.zeros((dim))
        self.var_running = torch.ones((dim))
        
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True) # batch variance
        else:
            xmean = self.mean_running
            xvar = self.var_running
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        # update the running mean and var (i.e. buffers in pytorch language)
        with torch.no_grad(): # not using gradient descent for running mean and var
            self.mean_running = self.momentum * xmean + (1 - self.momentum) * self.mean_running
            self.var_running = self.momentum * xvar + (1 - self.momentum) * self.var_running
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]


torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn((32, 100))
x = module(x)
x.shape # batch size 32, 100 features

torch.Size([32, 100])

In [3]:
x[:, 0].mean(), x[:, 0].std()
# so here each batch is normalized which means individual columns are normalized to 0 mean and 1 std

(tensor(1.4901e-08), tensor(1.0000))

In [5]:
# layer normalization is similar but instead of normalizing each column, it normalizes each row
# also remove the momentum and the running mean and var 
# also remove the training flag since we dont need to distinguish between training and inference 
import torch
class LayerNorm1d:
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.gamma = torch.ones((dim))
        self.beta = torch.zeros((dim))
        
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # instead of taking the mean of the entire batch, take the mean of each row
        xvar = x.var(1, keepdim=True) # instead of taking the variance of the entire batch, take the variance of each row
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

In [6]:
# after change, the columns are not normalized, but rows are
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn((32, 100))
x = module(x)

print(x[:, 0].mean(), x[:, 0].std())
print(x[0].mean(), x[0].std())

tensor(0.1469) tensor(0.8803)
tensor(2.3842e-09) tensor(1.0000)


- note: the transforer architecture has had one change in recent years
- the add+norm layer is done before the attention (transformation) layer (this is called pre-norm formulation)
- add dropout layer in various places to prevent overfitting, randoly drop the nodes at each training to regularize the model